In [6]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium 
import branca

def color_elev(elevation):  #changing color based on elevation
    if elevation < -3000:
        return "#100882"
    elif -3000 < elevation <= -1000:
        return "#1569ed"
    elif -1000 <= elevation < 1000:
        return "#0e8208"
    elif 1000 <= elevation < 3000:
        return "#f28313"
    elif elevation > 3000:
        return "#e30e07"  

# getting the data
dfs = pd.read_html('https://www.ngdc.noaa.gov/nndc/struts/results?type_0=Like&query_0=&op_8=eq&v_8=&type_10=EXACT&query_10=None+Selected&le_2=&ge_3=&le_3=&ge_2=&op_5=eq&v_5=&op_6=eq&v_6=&op_7=eq&v_7=&t=102557&s=5&d=5')
df = dfs[2]


In [7]:
# Correcting incorrectly named data entries 
df.replace(to_replace="Maars", value="Maar", inplace=True)
df.replace(to_replace="Lava domes", value="Lava dome", inplace=True)
df.replace(to_replace="Fissure vents", value="Fissure vent", inplace=True)
df.replace(to_replace="Stratovolcanos", value="Stratovolcano", inplace=True)
df.replace(to_replace="Stratovolcanoes", value="Stratovolcano", inplace=True)
df.replace(to_replace="Submarine volcano?", value="Submarine volcano", inplace=True)
df.replace(to_replace="Submarine volcanoes", value="Submarine volcano", inplace=True)
df.replace(to_replace="Shield volcanoes", value="Shield volcano", inplace=True)
df.replace(to_replace="Pyroclastic cones", value="Pyroclastic cone", inplace=True)
df.replace(to_replace="Scoria cones", value="Scoria cone", inplace=True)
df.replace(to_replace="Cinder cones", value="Cinder cone", inplace=True)


In [8]:
#declaring the data variables
v_name = df["Volcano Name"]
v_type = df["Type"]
lat = df["Latitude"]
lon = df["Longitude"]
elev = df["Elev"]

In [9]:
# defining the folium map
m = folium.Map(zoom_start=6) 

#defining all feature groups variables
Shield_volcano = folium.FeatureGroup(name="Shield volcano")
Stratovolcano = folium.FeatureGroup(name="Stratovolcano")
Maar = folium.FeatureGroup(name="Maar")
Volcanic_field = folium.FeatureGroup(name="Volcanic field")
Submarine_volcano = folium.FeatureGroup(name="Submarine volcano")
Caldera = folium.FeatureGroup(name="Caldera")
Complex_volcano = folium.FeatureGroup(name="Complex volcano")
Fissure_vent = folium.FeatureGroup(name="Fissure vent")
Compound_volcano = folium.FeatureGroup(name="Compound volcano")
Cinder_cone = folium.FeatureGroup(name="Cinder cone")
Pyroclastic_shield = folium.FeatureGroup(name="Pyroclastic shield")
Unknown = folium.FeatureGroup(name="Unknown")
Pyroclastic_cone = folium.FeatureGroup(name="Pyroclastic cone")
Scoria_cone = folium.FeatureGroup(name="Scoria cone")
Lava_dome = folium.FeatureGroup(name="Lava dome")
Lava_cone = folium.FeatureGroup(name="Lava cone")
Pumice_cone = folium.FeatureGroup(name="Pumice cone")
Crater_rows = folium.FeatureGroup(name="Crater rows")
Explosion_crater = folium.FeatureGroup(name="Explosion crater")
Tuff_rings = folium.FeatureGroup(name="Tuff rings")
Tuff_cone = folium.FeatureGroup(name="Tuff cone")
Somma_volcano = folium.FeatureGroup(name="Somma volcano")
Subglacial_volcano = folium.FeatureGroup(name="Subglacial volcano")
Fumarole_field = folium.FeatureGroup(name="Fumarole field")
Cone = folium.FeatureGroup(name="Cone")
Volcanic_complex = folium.FeatureGroup(name="Volcanic complex")
Hydrothermal_field = folium.FeatureGroup(name="Hydrothermal field")
Mud_volcano = folium.FeatureGroup(name="Mud volcano")
Not_Volcanic = folium.FeatureGroup(name="Not Volcanic")

# defining the dictionary
dict1 = {'Shield volcano':Shield_volcano, 'Stratovolcano':Stratovolcano, 'Maar':Maar, 'Volcanic field':Volcanic_field,
       'Submarine volcano':Submarine_volcano, 'Caldera':Caldera, 'Complex volcano':Complex_volcano, 'Fissure vent':Fissure_vent,
       'Compound volcano':Compound_volcano, 'Cinder cone':Cinder_cone, 'Pyroclastic shield':Pyroclastic_shield, 'Unknown':Unknown,
       'Pyroclastic cone':Pyroclastic_cone, 'Scoria cone':Scoria_cone, 'Lava dome':Lava_dome, 'Lava cone':Lava_cone,
       'Pumice cone':Pumice_cone, 'Crater rows':Crater_rows, 'Explosion crater':Explosion_crater, 'Tuff rings':Tuff_rings,
       'Tuff cone':Tuff_cone, 'Somma volcano':Somma_volcano, 'Subglacial volcano':Subglacial_volcano,
       'Fumarole field':Fumarole_field, 'Cone':Cone, 'Volcanic complex':Volcanic_complex, 'Hydrothermal field':Hydrothermal_field,
       'Mud volcano':Mud_volcano, 'Not Volcanic':Not_Volcanic}

# adding data points to the feature groups
for V_type,name,la,lo,el in zip(v_type,v_name,lat,lon,elev): 
    dict1[V_type].add_child(folium.CircleMarker(location=(la,lo),popup="Name:"+name+" "+"Elevation:"+str(el)+" m"+" "+"Type:"+V_type,radius=2.5 ,fill_color=color_elev(el),color=color_elev(el),fill_opacity=1))
    
# adding the feature groups to the map
for t in df.Type.unique():
    m.add_child(dict1[t])

#adding the map legend
colormap = branca.colormap.StepColormap(colors = ["#100882","#1569ed","#0e8208","#f28313","#e30e07"],index =  [-7000,-3000.0,-1000.0, 1000.0, 3000.0, 7000.0],vmin = -7000,vmax = 7000,caption='Elevation (Meter)').add_to(m)

#adding the map title 
title_html = '''<h3 align="center" style="font-size:20px"><b>World Volcano Data</b></h3><p align="center">Use Layer control in the top right corner of the map to select the type of volcano or click on the data point</p>'''
m.get_root().html.add_child(folium.Element(title_html))

#adding layer control for diffrent types of volcanos
m.add_child(folium.LayerControl())

#saving the map in a html file
m.save("output.html")

m

In [10]:
# readme - 
# World-Volcano-Data

# This project was an attempt to explore the location and different types of volconos in the world.

# The data for this project was extracted using web scraping library "BeautifulSoup" & "requests" from the website of National Oceanic and Atmospheric Administration(NOAA). Data source - https://rb.gy/utmee5

# The geographical map for this project was made using the library "folium".

# The data points are colored according to the elevation of that volcano. Color for the corresponding elevation can be refered through the map legend which is added using the "branca" library.

# Some data entries for type of volcano are misspelled, which are corrected in the code.

# Layer control feature is added to select/deselect the different types of volcanos like "shield volcano","strato volcano",etc.